Farmer problem (CVaR policy) 

Mauricio Tombini, Jordan Jalving, and Victor M. Zavala 2016 

University of Wisconsin-Madison 

In [6]:
using JuMP 
using Ipopt 

In [7]:
# Model parameters

NS = 5;                    # number of scenarios
S = collect(1:NS);       # scenario set
P = collect(1:3);        # set of crops (1=wheat,2=corn,3=beets)

# Data 
prcost = zeros(3)          # production (planting) cost
prcost[1] = 150;
prcost[2] = 230;
prcost[3] = 260;

pcost = zeros(3)           # purchase cost
pcost[1] = 238;
pcost[2] = 210;
pcost[3] = 0;

scost = zeros(3)           # sales cost
scost[1] = 170;
scost[2] = 150;
scost[3] = 36;

demand = zeros(3)          # demand
demand[1] = 200;
demand[2] = 240;
demand[3] = 0;

# assign random data

yield = zeros(length(S),3) # yields
yield[S,1] = 2.5;
yield[S,2] = 3.0;
yield[1,3] = 10;
yield[2,3] = 15;
yield[3,3] = 20;
yield[4,3] = 25;
yield[5,3] = 30;

In [8]:
# set cvar probability level
alpha = 4/5;              

In [9]:
# Model 
m = Model(solver=IpoptSolver(print_level=0))

@variable(m, x[S,P] >= 0)    # acres devoted to crops
@variable(m, y[S,P] >= 0)    # crops purchase
@variable(m, w[S,P] >= 0)    # crops sold;
@variable(m, phi[S] >= 0)    # cvar auxiliary variable
@variable(m, VaR)            # cvar auxiliary variable
@expression(m, Cost[s in S], sum(prcost[j]*x[s,j] + pcost[j]*y[s,j] - scost[j]*w[s,j] for j in P))
@variable(m, cost[s in S])

@constraint(m, varcost[s in S], cost[s] == Cost[s]) 
@constraint(m, cap[s in S], sum(x[s,j] for j in P) <= 500)
@constraint(m, bal[s in S,j in P], yield[s,j]*x[s,j]+y[s,j]-w[s,j] >= demand[j])
@constraint(m, sellb[s in S], w[s,3] <= 6000)
@constraint(m, buyb[s in S], y[s,3] <= 0)
@constraint(m, nonant[s in S,j in P], x[1,j] == x[s,j])
@constraint(m, cvar[s in S], cost[s]-VaR <= phi[s])

@objective(m, Min, VaR + (1/NS)*sum((1/alpha)*phi[s] for s in S))
 
solve(m)

:Optimal

In [10]:
# Results
println(getvalue(x))
println("")
println(getvalue(y))
println("")
println(getvalue(w))
println("")
println(getvalue(cost))
println("")
println("obj: ", getobjectivevalue(m))

x: 2 dimensions:
[1,:]
  [1,1] = 180.00000339993315
  [1,2] = 79.99999920014099
  [1,3] = 240.00000239988015
[2,:]
  [2,1] = 180.00000339993332
  [2,2] = 79.99999920014068
  [2,3] = 240.0000023998803
[3,:]
  [3,1] = 180.00000339993343
  [3,2] = 79.99999920014032
  [3,3] = 240.0000023998804
[4,:]
  [4,1] = 180.00000339992155
  [4,2] = 79.99999920014078
  [4,3] = 240.00000239989177
[5,:]
  [5,1] = 180.00000339994418
  [5,2] = 79.99999920014328
  [5,3] = 240.00000239986792

y: 2 dimensions:
[1,:]
  [1,1] = 0.0
  [1,2] = 0.0
  [1,3] = 0.0
[2,:]
  [2,1] = 0.0
  [2,2] = 0.0
  [2,3] = 0.0
[3,:]
  [3,1] = 0.0
  [3,2] = 0.0
  [3,3] = 0.0
[4,:]
  [4,1] = 0.0
  [4,2] = 0.0
  [4,3] = 0.0
[5,:]
  [5,1] = 0.0
  [5,2] = 0.0
  [5,3] = 0.0

w: 2 dimensions:
[1,:]
  [1,1] = 250.00001048992127
  [1,2] = 0.0
  [1,3] = 2400.000023998523
[2,:]
  [2,1] = 250.0000104899217
  [2,2] = 0.0
  [2,3] = 3600.0000359979263
[3,:]
  [3,1] = 250.00001048992198
  [3,2] = 0.0
  [3,3] = 4800.0000479973305
[4,:]
  [4,1] = 2